In [ ]:
import pandas as pd
import random

In [ ]:
issues = [
    "Login issue",
    "Payment failed",
    "Account blocked",
    "Refund delayed",
    "Card not working",
    "Transaction not reflected"
]


In [ ]:
channels = ["Chat", "Email", "Call"]
priorities = ["Low", "Medium", "High"]

In [ ]:
data = []


In [ ]:
for i in range(300):
    data.append({
        "Ticket_ID": i+1,
        "Issue_Text": random.choice(issues),
        "Channel": random.choice(channels),
        "Priority": random.choice(priorities),
        "Resolution_Time": random.randint(1,5)
    })


In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:


print(df.head())
print(df.info())


In [ ]:
print(df['Issue_Text'].value_counts().head(10))

In [ ]:
print(df['Priority'].value_counts())

In [ ]:
import re


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    return text

In [ ]:
df['clean_text'] = df['Issue_Text'].apply(clean_text)

In [ ]:
#sentiment analysis

In [ ]:
pip install textblob


In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

df['sentiment'] = df['clean_text'].apply(get_sentiment)


In [ ]:
#identifying high-risk issues

In [ ]:
def risk_level(row):
    if row['Priority'] == 'High' and row['sentiment'] < 0:
        return "High Risk"
    elif row['Priority'] == 'Medium':
        return "Medium Risk"
    else:
        return "Low Risk"

df['Risk_Level'] = df.apply(risk_level, axis=1)


In [ ]:
print(df['Risk_Level'].value_counts())


In [ ]:
#visualising

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df['Risk_Level'].value_counts().plot(kind='bar')
plt.title("Risk Distribution of Customer Issues")
plt.show()

In [ ]:
df.to_csv("final_ai_insights.csv", index=False)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['clean_text'])


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
df['Cluster'] = kmeans.fit_predict(X)


In [ ]:
print(df.groupby('Cluster')['Issue_Text'].head(3))


In [ ]:
summary_data = df.groupby('Cluster').agg({
    'Issue_Text':'count',
    'sentiment':'mean'
}).reset_index()

summary_data.columns = ['Cluster','Total_Issues','Avg_Sentiment']

print(summary_data)


In [ ]:
def generate_insight(row):
    if row['Avg_Sentiment'] < 0:
        tone = "negative customer sentiment"
    else:
        tone = "neutral or positive sentiment"

    return f"Cluster {row['Cluster']} has {row['Total_Issues']} issues showing {tone}, indicating potential process improvement opportunities."

summary_data['AI_Insight'] = summary_data.apply(generate_insight, axis=1)

print(summary_data[['Cluster','AI_Insight']])


In [ ]:
high_risk = df[df['Risk_Level']=="High Risk"].shape[0]
total = df.shape[0]

executive_summary = f"""
AI Analysis Summary:
- Total issues analyzed: {total}
- High-risk issues detected: {high_risk}
- Major patterns were automatically identified using NLP clustering.
- Key recommendation: Focus on clusters showing negative sentiment trends.
"""

print(executive_summary)


In [ ]:
#ai assistant

In [ ]:
!pip install transformers


In [ ]:
from transformers import pipeline

ai_assistant = pipeline("text-generation", model="gpt2")


In [ ]:
context = summary_data.to_string()

def ask_ai(question):
    prompt = f"""
You are an AI Operations Analyst.

Here is the analysis data:
{context}

Question: {question}

Answer:
"""

    response = ai_assistant(
        prompt,
        max_length=200,
        num_return_sequences=1,
        temperature=0.7
    )

    return response[0]['generated_text']


In [ ]:
print(ask_ai("What is the biggest risk area?"))
